In [1]:
import cv2
import numpy as np

# YOLOv3 ağırlık dosyası ve yapılandırma dosyası
weights_path = "yolov3.weights"
config_path = "yolov3.cfg"

# Sınıf etiketlerinin bulunduğu dosya
labels_path = "coco.names"

# YOLOv3 ağırlıklarını ve yapılandırmayı yükleme
net = cv2.dnn.readNetFromDarknet(config_path, weights_path)

# Sınıf etiketlerini yükleme
with open(labels_path, "r") as f:
    labels = f.read().splitlines()

# Kamera veya video dosyasını başlatma
video_path = "video.mp4"
cap = cv2.VideoCapture(video_path)

# Gerçek zamanlı nesne tanıma döngüsü
while True:
    # Kareden çerçeve alma
    ret, frame = cap.read()

    if not ret:
        break

    # Girdi çerçevesi boyutlarını alma
    height, width, _ = frame.shape

    # Girdi çerçevesi için bir blob oluşturma
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)

    # YOLOv3 ağı üzerinde ileri geçme
    net.setInput(blob)
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    outputs = net.forward(output_layers)

    # Algılanan nesneleri saklama
    class_ids = []
    confidences = []
    boxes = []

    # Çıktıları işleme
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5:  # Güven eşiği
                # Nesnenin sınıf etiketi ve konumunu alma
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                # Algılanan nesnelerin bilgilerini saklama
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])

    # Non-Maximum Suppression uygulama
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Algılanan nesneleri çerçeveye çizme
    font = cv2.FONT_HERSHEY_SIMPLEX
    colors = np.random.uniform(0, 255, size=(len(labels), 3))

    if len(indexes) > 0:
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            label = str(labels[class_ids[i]])
            confidence = confidences[i]
            color = colors[class_ids[i]]

            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, f"{label}: {confidence:.2f}", (x, y - 10), font, 0.5, color, 2)

    # Çıktıyı gösterme
    cv2.imshow("YOLOv3 Object Detection", frame)

    # 'q' tuşuna basılınca çıkış
    if cv2.waitKey(1) == ord('q'):
        break

# Kaynakları serbest bırakma
cap.release()
cv2.destroyAllWindows()
